# Peaks Over Threshold (POT) - Poisson & Generalized Pareto distributions

Waves database - Significant Wave Height

* [1. Input Hs](#1)
	* [1.1. Load Dataset](#11)  
	* [1.2. Data Visualization: Time Series](#12)     
	* [1.3. Data Visualization: Histograms](#13)     
    
* [2. Peaks Over Threshold (POT)](#2)
	* [2.1. Calculate POT](#21)
    * [2.2. Data visualization - Hourly and POT Hs](#22)

* [3. Fit POT Frequency and Intensity](#3)
	* [3.1 Fit POT Frequency to Poisson disribution](#31)
	* [3.2 Fit POT Intensity to Generalized Pareto disribution](#32)
    
* [4. Simulate Precipitation Extremes](#4)
	* [4.1. Use Frequency and Intensity distributions (Poisson and Generalized Pareto) to generate Hs Extremes](#41)
    * [4.2. Plot return period](#42)
    
<hr size="5"/>

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import os
import os.path as op
import sys 

# arrays
import numpy as np
import pandas as pd
import xarray as xr
import datetime
from numpy.random import multivariate_normal
from scipy.stats import poisson, genpareto, genextreme
from scipy.optimize import curve_fit
from scipy.io import loadmat

# matplotlib
import matplotlib.pyplot as plt

# plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 1. Input Hs <a class="anchor" id="1"></a>

### 1.1. Load Dataset <a class="anchor" id="11"></a>

In [ ]:
# path
p_db = op.join(os.getcwd(),'..','..','data','Waves')

# read database precipitation (xls file)
p_dat = op.join(p_db, 'cantabria_CSIRO.mat')

data_mat = loadmat(p_dat)
data_raw = pd.DataFrame(
    {
        'hs':data_mat['data']['hs'][0][0].squeeze(),
        'year':data_mat['data']['year'][0][0].squeeze(),
        'month':data_mat['data']['month'][0][0].squeeze(),
        'day':data_mat['data']['day'][0][0].squeeze(),
        'hour':data_mat['data']['hour'][0][0].squeeze(),
    }
)

data = pd.DataFrame({
    'time': pd.to_datetime(data_raw[['year', 'month', 'day', 'hour']]),
    'hs': data_raw['hs'].squeeze()}
)
data.set_index('time', inplace=True)

data


### 1.2. Data Visualization: Time Series <a class="anchor" id="12"></a>

In [ ]:
px.line(data)

### 1.3. Data Visualization - Histograms <a class="anchor" id="13"></a>

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Count', 'Probability Density'],
)
fig.append_trace(
    go.Histogram(x = data['hs'], nbinsx = 30), 
    1, 1,
)
fig.append_trace( 
    go.Histogram(x = data['hs'], nbinsx = 30, histnorm='probability density'), 
    1, 2
)
fig.update_layout(showlegend=False)
fig.show()


## 2. Peaks Over Threshold (POT) 

### 2.1. Calculate POT 

In [ ]:
# Set threshold 
th = np.percentile(data.hs, 98) 
ie = 5   # days between independent peaks

# Calculate data over threshold
data_pot = data.loc[data['hs'] >= th]

# Days between consecutive events
ix_space = np.diff(data_pot.index).astype('timedelta64[h]').astype(int)
ix_space = list(ix_space) + [ie] # include last event

# Get independent peaks
idx = []
for nt in np.where(ix_space)[0]:
    day = data_pot.index[nt]
    rangeday = np.arange(day-datetime.timedelta(days=ie), day+datetime.timedelta(days=ie), datetime.timedelta(hours=1))
    if data_pot.iloc[nt].values == np.max(data.loc[rangeday]).values:
        idx.append(nt)

data_pot = data_pot.iloc[idx]


In [ ]:
# Plot Precipitation extreme time series
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = data.index, y = data['hs'],
        mode ='lines', name = 'Historical Hs',
    )
)
fig.add_trace(
    go.Scatter(
        x = data_pot.index, y = data_pot['hs'],
        mode ='markers', name = 'Peaks Over Threshold',
    )
)
fig.add_shape(
    type="line",
    x0=0, x1=1, xref='paper',
    y0=th,  y1=th, yref='y',
    line=dict(
        color="gray",
        dash="dash",
    )
)
fig.update_layout(    
    xaxis_title = "time",
    yaxis_title = "Hs (m)",
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


### 2.2. Data visualization - Hourly and POT precipitation 

In [ ]:
# extreme series 
var_max = data_pot['hs'].values[:] 

fig = go.Figure()
fig.add_trace( 
    go.Histogram(x = data['hs'],  nbinsx = 100, histnorm='probability density', name='Hourly Hs')
)

fig.add_trace(
    go.Histogram(x=var_max,  nbinsx = 100, histnorm='probability density', name='Peaks Over Threshold')
)

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Probability Density Functions',
    showlegend=True, 
    barmode='overlay'
)
fig.show()


## 3. Fit POT Frequency and Intensity

### 3.1 Fit POT Frequency to Poisson disribution

In [ ]:
# get number of events each year 
y_events = data_pot.groupby(by=[data_pot.index.year]).count().hs.values

# get frequency
N, freq = np.unique(y_events, return_counts=True)

# optimize poisson lambda parameter
params = np.mean(y_events)
print('lambda Poisson : {0}'.format(np.float(params)))

# freeze poisson distribution
rv_poi = poisson(params)

# generate some values from poisson to plot
x = np.arange(rv_poi.ppf(0), rv_poi.ppf(0.999))
y = rv_poi.pmf(x) 

In [ ]:
# Plot POISSON PMF vs data frequency histogram
print(freq)

fig = go.Figure()
fig.add_trace(go.Histogram(x=y_events, histnorm='probability density', name='Historical Hs'))
fig.add_trace(go.Bar(x=x, y=y, name='PMF'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Poisson Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


In [ ]:
# simulate some values for this Poisson
size_sim = 1000
var_sim = rv_poi.rvs(size=size_sim)

# Plot POI pdf vs simulated data
fig = go.Figure()
fig.add_trace(go.Histogram(x=var_sim, histnorm='probability density', name='Simulation Hs'))
fig.add_trace(go.Bar(x=x, y=y, name='PMF'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Poisson Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


### 3.2 Fit POT Intensity to Generalized Pareto disribution 

In [ ]:
# get event max intensity (for hydrologic year)
imax = data_pot.groupby(by=[data_pot.index.year]).max()
intensity = data_pot.values - th

# fit data to GPD
shape, loc, scale = genpareto.fit(intensity, floc=0)
print(shape, loc, scale)

# negative loglikelihood
nLogL = genpareto.nnlf((shape, loc, scale), intensity)

# GPD parameters
theta = (shape, loc, scale)

# freeze GPD with parameters, get GPD PDF
rv_gpd = genpareto(shape, loc, scale)  
x = np.linspace(rv_gpd.ppf(0), rv_gpd.ppf(1), 1000)
y = rv_gpd.pdf(x)


In [ ]:
# Plot GPD PDF vs data probability density histogram

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='PDF', marker_color='black'))
fig.add_trace(go.Histogram(x=intensity.reshape(-1), nbinsx=100,histnorm='probability density', name='Historical'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'GPD Probability Density Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.update_xaxes(range=[0, x.max()])
fig.show()

In [ ]:
# simulate some values for this GPD
var_sim = rv_gpd.rvs(size=size_sim)

# Plot POI pdf vs simulated data
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='PDF', marker_color='black'))
fig.add_trace(go.Histogram(x=var_sim, histnorm='probability density', name='Simulation'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Generalized Pareto Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.update_xaxes(range=[0, x.max()])
fig.show()

## 4. Simulate Hs Extremes 

### 4.1. Use Frequency and Intensity distributions (Poisson and Generalized Pareto) to generate Hs Extremes

In [ ]:
# years of Hs extremes to simulate
n_sims = 1000
years_sim = 100 

ds_sim = xr.Dataset()

for c, ts in enumerate(range(n_sims)):
    
    sys.stdout.write('Simulation {0} from {1}\r'.format(c, len(range(n_sims))))
    sys.stdout.flush()
    
    # empty array to storage annual maximas
    annual_max = []
    
    for y in range(years_sim):
        
        n_events = rv_poi.rvs(size=1) #one random value
        events = genpareto.rvs(shape, loc=0, scale=scale, size=n_events)
        
        if n_events >0:
            annual_max.append(np.nanmax(events) + th)
        else:
            annual_max.append(th)

    # Simulated Hs Extremes
    sim_extremes = pd.DataFrame({'hs' : annual_max})
    sim_extremes.index.name = 'year'
    sim_extremes = sim_extremes.to_xarray().assign_coords(sim=c).expand_dims('sim')

    ds_sim = xr.merge([sim_extremes, ds_sim])


### 4.2. Plot return period

In [ ]:
# aux func for calculating rp time
def t_rp(time_y):
    ny = len(time_y)
    return np.array([1/(1-(n/(ny+1))) for n in np.arange(1,ny+1)])

# historical rp time and sorted annual maxima
trp_hist = t_rp(imax.index)
trp_hist_val = np.sort(imax['hs'])

# simulation rp time and sorted annual maxima
trp_sim = t_rp(ds_sim.year.values)
trp_sim_val = np.sort(ds_sim.hs.values)

# calculate simulation maxima percentiles
p95 = np.nanpercentile(trp_sim_val, 100-5/2.0, axis=0,)
p50 = np.nanpercentile(trp_sim_val, 50, axis=0,)
p05 = np.nanpercentile(trp_sim_val, 5/2.0, axis=0,)

In [ ]:
# Plot return period

fig = go.Figure()
fig.add_trace(go.Scatter(x=trp_sim, y=p95, mode='lines', name='P95', marker_color='mediumturquoise'))
fig.add_trace(go.Scatter(x=trp_sim, y=p05, mode='lines', name='P05', marker_color='mediumturquoise',  fill='tonexty', fillcolor='rgba(0, 181, 204, 0.10)'))
fig.add_trace(go.Scatter(x=trp_sim, y=p50, mode='lines', name='P50', marker_color='black'))
fig.add_trace(go.Scatter(x=trp_hist, y=trp_hist_val, mode='markers', name='Hist', marker_color='red'))


fig.update_xaxes(type="log")
fig.update_layout(    
    xaxis_title = "Return Period (years)",
    yaxis_title = "Hs (m)",
    title = 'POT',
    width=400*2.5, height=300*2.5
)
fig.show()